In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from torchmetrics import StructuralSimilarityIndexMeasure
from statistics import median, mean
from matplotlib import pyplot as plt
import numpy as np
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
from torchsummary import summary
import json
from tqdm import tqdm
from glob import glob
import sys
path = "/study/mrphys/skunkworks/kk/mriUnet"
sys.path.insert(0,path)
import unet
from torchvision import transforms
from torch.utils.data import Dataset
import h5py
from sklearn.model_selection import KFold as kf
import os

allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))

In [9]:
def getComplexSlices(path):

    with h5py.File(path,'r') as hf:
        prefix = 'C_000_0'
        imagestackReal = []
        imagestackImag = []
        for i in range(10):
            n = prefix + str(i).zfill(2)
            image = hf['Images'][n]
            imagestackReal.append(np.array(image['real']))
            imagestackImag.append(np.array(image['imag']))
            if i==0:
                normScale = np.abs(np.array(image['real']+image['real']*1j)).max()
        imagestackReal = np.array(imagestackReal)/normScale
        imagestackImag = np.array(imagestackImag)/normScale
        
    return imagestackReal+imagestackImag*1j

class mriSliceDataset(Dataset):
    def __init__(self, sample):
        self.originalPath = []
        self.accelPath = [] 

        allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))
        folderName  = allImages[sample]
        self.originalPath = folderName + 'processed_data/C.h5'
        self.accelPath = folderName +'processed_data/acc_2min/C.h5'
        
        self.originalFile = getComplexSlices(self.originalPath)
        self.accelFile = getComplexSlices(self.accelPath)

    def __getitem__(self, index):
        if index<256:
            return self.accelFile[index,:,:], self.originalFile[index,:,:]
        elif index<512:
            return self.accelFile[:,index,:], self.originalFile[:,index,:]
        else:
            return self.accelFile[:,:,index], self.originalFile[:,:,index]
        
    def __len__(self):
        return 768

In [14]:
for i in tqdm(range(len(allImages))):
    dataset = mriSliceDataset(i)
    with open(f'/scratch/mrphys/pickled/dataset_{i}.pickle', 'wb') as f:
        pickle.dump(dataset, f)

100%|███████████████████████████████████████████████████████████████████████████████████| 65/65 [14:16<00:00, 13.17s/it]


In [3]:
def getComplexSlices(path):

    with h5py.File(path,'r') as hf:
        prefix = 'C_000_0'
        imagestackReal = []
        imagestackImag = []
        for i in range(10):
            n = prefix + str(i).zfill(2)
            image = hf['Images'][n]
            normScale = np.abs(np.array(image['real']+image['imag']*1j)).max()
            imagestackReal.append(np.array(image['real'])/normScale)
            imagestackImag.append(np.array(image['imag'])/normScale)
        imagestackReal = np.array(imagestackReal)
        imagestackImag = np.array(imagestackImag)
        
    return imagestackReal+imagestackImag*1j

class mriSliceDataset(Dataset):
    def __init__(self, sample):
        self.originalPath = []
        self.accelPath = [] 

        allImages = sorted(glob("/study/mrphys/skunkworks/training_data//mover01/*/", recursive=True))
        folderName  = allImages[sample]
        self.originalPath = folderName + 'processed_data/C.h5'
        self.accelPath = folderName +'processed_data/acc_2min/C.h5'
        
        self.originalFile = getComplexSlices(self.originalPath)
        self.accelFile = getComplexSlices(self.accelPath)

    def __getitem__(self, index):
        if index<256:
            return self.accelFile[index,:,:], self.originalFile[index,:,:]
        elif index<512:
            return self.accelFile[:,index,:], self.originalFile[:,index,:]
        else:
            return self.accelFile[:,:,index], self.originalFile[:,:,index]
        
    def __len__(self):
        return 768
    
for i in tqdm(range(len(allImages))):
    dataset = mriSliceDataset(i)
    with open(f'/scratch/mrphys/pickled/ddpm_dataset_{i}.pickle', 'wb') as f:
        pickle.dump(dataset, f)

100%|███████████████████████████████████████████████████████████████████████████████████| 65/65 [43:17<00:00, 39.96s/it]
